In [3]:
# Used to store locations in the configuration space
type Point
   x::Float64
   y::Float64
   z::Float64
   p::Float64
end

type poly_segment
# Coefficients of the polynomial
   x_coeffs::Vector{Float64}
   y_coeffs::Vector{Float64}
   z_coeffs::Vector{Float64}
   p_coeffs::Vector{Float64}
   t::Float64 # Time of the polynomial segment
   q::Float64 # Cost of segment
   cells::Vector{Int64} # List of points which are spaced to reduce collision check times.
   init_config::Vector{Point} # Initial position, velocity, acceleration
   final_config::Vector{Point} # Final position, velocity, acceleration
end


#Function evaluate_poly evaluates a given derivative of a polynomial at a certain value
#Assumptions
# One dimensional polynomial
#Inputs
# coeffs - a vector of the polynomial coefficients that form the polynomial from c1 + c2*t + c3*t^2 + ...
# deriv - the number representing the ith derivative of the polynomial, a value of 0 means the 0th derivative
# t - the value(s) at which the derivative is evaluated at
#Outputs
# x - the value of the polynomial
function evaluate_poly(coeffs::Vector{Float64}, deriv::Int64, t)
    #see how many coefficients there are
    p = length(coeffs);
    #initialize x to zero
    x = 0;
    #loop as many times are needed to go through the polynomial after it has been derived
    for e = 0:(p-deriv-1)
        #the looping variable will start at zero since the polynomial's first power will start with at 0 no matter what
        #create the coefficient indexer by adding one to the looping variable and andding the deriv number
        ci = e + deriv +1;
        #calculate the derivative caused constants by using factorials
        d = factorial(e + deriv) / factorial(e);
        #combine the terms through multiplication and add to x
        x += d*coeffs[ci]*t.^e
        #repeat the loop
    end
    #return the end value
    return x;
end



#Function occupancyCellChecker determines offline what cells a path runs through in an occupancy grid and returns
# the unique cell IDs that the path goes through
#Assumptions
# An occupancy function exists that is called as follows ID = occupancy_get_id(x,y,z)
# Path starts at zero
# Solution has coefficient vectors of the same length
#Inputs/Needed Variables
# solution - an object containing points, and times related to path solution
# grid_resx - the resolution of the grid in the x direction
# grid_resy - the resolution of the grid in the y direction
# grid_resz - the resolution of the grid in the z direction
# dim - the dimension of the path to be checked
# aggressParam - a parameter of how aggressive to check a path (0,infty), Closer to 0 means check every point on the path, Closer to infinity => check no points
#Outputs
# occupancy_vec - a vector of the occupancy IDs that the polynomial is characterized by
function(solution::poly_segment, grid_resx::Float64, grid_resy::Float64, grid_resz::Float64, dim::Int64, aggressParam::Float64)
    #Check if the dimension is reasonable otherwise print error and exit;
    if(dim < 1 || dim > 3)
        println("Invalid dimension entered")
        return -1;
    end

    #Read in the coefficients into a matrix
    coeffMat = [solution.x_coeffs; solution.y_coeffs; solution.z_coeffs];
    #Create a holder for derivatives, delta_t's, and pts when we get there in the for loop
    derivMat = zeros(dim);
    delta_t = zeros(dim);
    pts = zeros(dim);
    #Initialize/read in the resolution variables if needed
    #grid_resx = 0.05;
    #grid_resy = 0.05;
    #grid_resz = 0.05;
    #Initialize times assuming polynomials are solve at 0 time initial
    t = 0;
    timeFin = solution.t[end] - solution.t[1];
    #Calculate the distance time to travel in each direction before getting occupancy grid id using 1st order taylor series approximation
    dist_x = grid_resx * agreessParam;
    dist_y = grid_resy * agreessParam;
    dist_z = grid_resz * agreessParam;
    #Put into a matrix
    dist_to_travel = [dist_x; dist_y; dist_z]
    #Loop for 2 or 3 dimensions if dim >3
    for looper = 1:dim
        #evaluate the polynomial first direvative at current time
        derivMat[looper] = evaluate_poly(coeffMat[looper, :], 1, t);
        #looper = 1 -> x, 2 -> y, 3 -> z
        #find the required delta_t's needed to go the set distance before checking occupancy
        delta_t[looper] = abs(dist_to_travel[looper] / derivMat[looper]);
    end

    #Create a counter
    counter =1;

    #Check if at the or past the end time and while loop if not passed it
    while(t < timeFin)
        #Find the smallest one
        changeDelta, deltaIndex = findmin(delta_t)
        #Move the time by the smallest one
        t += changeDelta;
        #Decrease the delta t's for the others
        for looper = 1:dim
            #Subract the smallest delta t from the other deltas
            delta_t[looper] -= changeDelta;
        end
        #Find the x,y,z of the current time
        for looper = 1:dim
            pts[looper] = evaluate_poly(coeffMat[looper, :], 0, t);
        end
        #Find occupancy ID at current point by adding to a vector
        occupancy_vec[counter] = occupancy_get_id(pts[1], pts[2], pts[3]);
        #Evaluate the new delta_t for the dimension
        delta_t[deltaIndex] = abs(dist_to_travel[deltaIndex]/evaluate_poly(coeffMat[deltaIndex, :], 1, t));
        #Increment counter
        counter += 1;
    end

    #Return the vector of unique occupancy grids
    return unique(occupanyc_vec);
end


(::#3) (generic function with 1 method)

Int64, Any) in module Main at In[1]:12 overwritten at In[3]:34.
